# Feature Retrieval
This notebook shows the basic of feature retrieval using Feast. The `retrieval.ipynb` notebook in `/integration_feature_store/feature_repo` shows more involved usage.

In [ ]:
# The following cell executes the script `run.py` which creates the Feast registry. Ignore the numpy warnings
!python run.py

from IPython.display import clear_output
clear_output()  # clears the jupyter cell output

In [ ]:
# View the list of feature-views in the Feast registry;
# if empty, make sure to execute `run.py` without failure
!feast feature-views list

In [ ]:
import datetime

import feast
from hamilton import base, driver

import feature_transformations
import store_definitions
import store_operations  # Feast operations as Hamilton DAG
import demo_inputs  # hard-coded inputs for demo

%load_ext autoreload
%autoreload 1
%aimport store_definitions, feature_transformations

Visualize the structure of the Feast repository

In [ ]:
dr = driver.Driver(
    dict(),
    # feature_transformations,
    store_definitions,
    adapter=base.SimplePythonGraphAdapter(base.DictResult())
)

dr.display_all_functions(
    "objs",
    render_kwargs={"format": "png", "view": False},
)

# Feast pattern
Feast relies on the FeatureStore object to connect to the feature store and interact with it using its methods. The FeatureStore already has a great interface. To use with Hamilton and gain the modularity, reusability, testability, etc. benefits you can usually wrap the FeatureStore in a Hamilton function (more on that later). To learn more about [Feast retrieval](https://docs.feast.dev/getting-started/concepts/feature-retrieval).

In [ ]:
# instantiate FeatureStore object
feature_store = feast.FeatureStore(repo_path=".")

## Offline / Historical
The following cell will retrieve historical data stored in an offline store. Therefore, it will access the Feast DataSource and do the joins at request time.

In [ ]:
# define a retrieval job for entities specified by id and timestamp 
# `features` specifies columns to retrieve; can be feast.FeatureView, 
# feast.FeatureService, or column names as string
job: feast.infra.offline_stores.offline_store.RetrievalJob = feature_store.get_historical_features(
    entity_df=demo_inputs.HISTORICAL_ENTITY_DF,
    features=demo_inputs.HISTORICAL_FEATURES,
)

df = job.to_df()
df.head()

## Online with materialization
The following cells show an online retrieval pattern. First, data from the offline store is materialized for the past `n_days`. This creates the joins and stores the FeatureViews in the online store. Then, features can be easily retrieved.

In [ ]:
n_days = 90
end_date = datetime.datetime.now()
start_date = end_date - datetime.timedelta(days=n_days)
feature_store.materialize(start_date=start_date, end_date=end_date)

In [ ]:
# load the FeatureService `driver_activity_v1`
service: feast.FeatureService = feature_store.get_feature_service("driver_activity_v1")
response: feast.online_response.OnlineResponse = feature_store.get_online_features(
    entity_rows=demo_inputs.ONLINE_ENTITY_ROWS,
    features=service,
)

df = response.to_df()
df.head()

# Hamilton pattern
With Hamilton, the different operations are defined as function once and stored in the module `store_operations.py`. The variable `final_vars` defines the operations to execute and `inputs` specifies the entities and features to query.

In [ ]:
dr = driver.Driver(
    dict(
        feast_repository_path=".",
        feast_config={},
        driver_source_path="./data/driver_stats.parquet",
        trips_raw_path="./data/trips_raw.parquet",
        trips_stats_3h_path="./data/trips_stats_3h.parquet",
    ),
    feature_transformations,
    store_operations,
    store_definitions,
)

final_vars = [
    "historical_features",  # specify the function defined in `store_operations`
]

inputs = dict(
    entity_df=demo_inputs.HISTORICAL_ENTITY_DF,
    historical_features_=demo_inputs.HISTORICAL_FEATURES,
    batch=True,  # triggers the `config.when(batch=True)` for historical retrieval
)

out = dr.execute(final_vars=final_vars, inputs=inputs)
out